# 01 - Data Exploration

In this notebook, I'm exploring the raw EPL data to understand what we're working with. I'll inspect the data quality and do some initial analysis.

## My Goals:
1. Load all season data files
2. Understand the data structure
3. Check for missing values
4. Identify Arsenal matches
5. Understand what data quality issues exist


## Step 1: Import Libraries

Importing the libraries I need:
- **pandas**: For data manipulation (like Excel for Python)
- **numpy**: For numerical operations
- **matplotlib & seaborn**: For visualizations
- **pathlib**: For handling file paths


In [29]:
# Import all the libraries I need
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Set random seed for reproducibility (so results are consistent)
np.random.seed(42)

# Set plotting style to make graphs look nice
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Display settings - show more rows/columns when printing
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', 100)      # Show up to 100 rows

print("Libraries imported successfully!")


Libraries imported successfully!


## Step 2: Define Data Path

Set the path to where our raw data files are stored.


In [30]:
# Define the path to raw data
RAW_DATA_DIR = Path('../data/raw')

# Check if the directory exists
if RAW_DATA_DIR.exists():
    print(f"Data directory found: {RAW_DATA_DIR}")
    
    # List all CSV files
    csv_files = list(RAW_DATA_DIR.glob('epl-*.csv'))
    print(f"\nFound {len(csv_files)} CSV files:")
    for file in sorted(csv_files):
        print(f"   - {file.name}")
else:
    print(f"Data directory not found: {RAW_DATA_DIR}")


Data directory found: ../data/raw

Found 26 CSV files:
   - epl-2000-01.csv
   - epl-2001-02.csv
   - epl-2002-03.csv
   - epl-2003-04.csv
   - epl-2004-05.csv
   - epl-2005-06.csv
   - epl-2006-07.csv
   - epl-2007-08.csv
   - epl-2008-09.csv
   - epl-2009-10.csv
   - epl-2010-11.csv
   - epl-2011-12.csv
   - epl-2012-13.csv
   - epl-2013-14.csv
   - epl-2014-15.csv
   - epl-2015-16.csv
   - epl-2016-17.csv
   - epl-2017-18.csv
   - epl-2018-19.csv
   - epl-2019-20.csv
   - epl-2020-21.csv
   - epl-2021-22.csv
   - epl-2022-23.csv
   - epl-2023-24.csv
   - epl-2024-25.csv
   - epl-2025-26.csv


## Step 3: Load a Sample File

I'm starting by loading just ONE file to understand the structure before loading all files.


In [31]:
# Load a recent season file (2024-25) as a sample
sample_file = RAW_DATA_DIR / 'epl-2024-25.csv'

if sample_file.exists():
    df_sample = pd.read_csv(sample_file)
    
    print(f"Loaded {sample_file.name}")
    print(f"\nShape: {df_sample.shape[0]} rows × {df_sample.shape[1]} columns")
    print(f"\nColumn names:")
    for i, col in enumerate(df_sample.columns, 1):
        print(f"   {i:2d}. {col}")
    
    print(f"\nFirst few rows:")
    display(df_sample.head())
    
    print(f"\nData types:")
    print(df_sample.dtypes)
    
    print(f"\nMissing values:")
    missing = df_sample.isnull().sum()
    missing_pct = (missing / len(df_sample) * 100).round(2)
    missing_df = pd.DataFrame({
        'Missing Count': missing,
        'Missing %': missing_pct
    })
    missing_df = missing_df[missing_df['Missing Count'] > 0].sort_values('Missing Count', ascending=False)
    if len(missing_df) > 0:
        display(missing_df)
    else:
        print("   No missing values!")
else:
    print(f"File not found: {sample_file}")


Loaded epl-2024-25.csv

Shape: 380 rows × 120 columns

Column names:
    1. Div
    2. Date
    3. Time
    4. HomeTeam
    5. AwayTeam
    6. FTHG
    7. FTAG
    8. FTR
    9. HTHG
   10. HTAG
   11. HTR
   12. Referee
   13. HS
   14. AS
   15. HST
   16. AST
   17. HF
   18. AF
   19. HC
   20. AC
   21. HY
   22. AY
   23. HR
   24. AR
   25. B365H
   26. B365D
   27. B365A
   28. BWH
   29. BWD
   30. BWA
   31. BFH
   32. BFD
   33. BFA
   34. PSH
   35. PSD
   36. PSA
   37. WHH
   38. WHD
   39. WHA
   40. 1XBH
   41. 1XBD
   42. 1XBA
   43. MaxH
   44. MaxD
   45. MaxA
   46. AvgH
   47. AvgD
   48. AvgA
   49. BFEH
   50. BFED
   51. BFEA
   52. B365>2.5
   53. B365<2.5
   54. P>2.5
   55. P<2.5
   56. Max>2.5
   57. Max<2.5
   58. Avg>2.5
   59. Avg<2.5
   60. BFE>2.5
   61. BFE<2.5
   62. AHh
   63. B365AHH
   64. B365AHA
   65. PAHH
   66. PAHA
   67. MaxAHH
   68. MaxAHA
   69. AvgAHH
   70. AvgAHA
   71. BFEAHH
   72. BFEAHA
   73. B365CH
   74. B365CD
   75. B365CA
   

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,BWH,BWD,BWA,BFH,BFD,BFA,PSH,PSD,PSA,WHH,WHD,WHA,1XBH,1XBD,1XBA,MaxH,MaxD,MaxA,AvgH,AvgD,AvgA,BFEH,BFED,BFEA,B365>2.5,B365<2.5,P>2.5,P<2.5,Max>2.5,Max<2.5,Avg>2.5,Avg<2.5,BFE>2.5,BFE<2.5,AHh,B365AHH,B365AHA,PAHH,PAHA,MaxAHH,MaxAHA,AvgAHH,AvgAHA,BFEAHH,BFEAHA,B365CH,B365CD,B365CA,BWCH,BWCD,BWCA,BFCH,BFCD,BFCA,PSCH,PSCD,PSCA,WHCH,WHCD,WHCA,1XBCH,1XBCD,1XBCA,MaxCH,MaxCD,MaxCA,AvgCH,AvgCD,AvgCA,BFECH,BFECD,BFECA,B365C>2.5,B365C<2.5,PC>2.5,PC<2.5,MaxC>2.5,MaxC<2.5,AvgC>2.5,AvgC<2.5,BFEC>2.5,BFEC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA,BFECAHH,BFECAHA
0,E0,16/08/2024,20:00,Man United,Fulham,1,0,H,0,0,D,R Jones,14,10,5,2,12,10,7,8,2,3,0,0,1.60,4.20,5.25,1.60,4.4,5.25,1.60,4.33,5.00,1.63,4.38,5.30,1.65,4.2,5.00,1.68,4.32,5.03,1.68,4.50,5.50,1.62,4.36,5.15,1.66,4.5,5.60,1.53,2.50,1.56,2.56,1.57,2.60,1.53,2.52,1.59,2.64,-1.0,2.05,1.88,2.07,1.86,2.07,1.89,2.03,1.85,2.10,1.88,1.67,4.10,5.00,1.65,4.20,4.80,1.62,4.0,5.00,1.65,4.23,5.28,1.60,4.2,5.50,1.66,4.15,5.33,1.70,4.33,5.50,1.66,4.20,5.02,1.72,4.20,5.40,1.62,2.30,1.63,2.38,1.66,2.45,1.61,2.37,1.68,2.46,-0.75,1.86,2.07,1.83,2.11,1.88,2.11,1.82,2.05,1.90,2.08
1,E0,17/08/2024,12:30,Ipswich,Liverpool,0,2,A,0,0,D,T Robinson,7,18,2,5,9,18,2,10,3,1,0,0,8.50,5.50,1.33,7.50,5.5,1.36,8.50,5.50,1.33,8.18,5.84,1.34,8.50,5.5,1.33,8.60,5.85,1.35,9.00,6.10,1.37,8.28,5.76,1.34,9.40,6.2,1.36,1.40,3.00,1.41,3.00,1.43,3.07,1.41,2.94,1.45,3.15,1.5,2.02,1.91,1.99,1.92,2.02,1.95,1.97,1.90,2.04,1.93,8.00,5.75,1.33,8.00,5.75,1.34,7.50,5.5,1.33,8.14,6.09,1.34,8.00,5.5,1.35,8.57,5.85,1.34,8.57,6.25,1.39,7.87,5.81,1.35,8.60,6.20,1.37,1.36,3.20,1.37,3.30,1.40,3.38,1.37,3.18,1.40,3.40,1.50,2.05,1.88,2.04,1.90,2.20,2.00,1.99,1.88,2.04,1.93
2,E0,17/08/2024,15:00,Arsenal,Wolves,2,0,H,1,0,H,J Gillett,18,9,6,3,17,14,8,2,2,2,0,0,1.18,7.50,13.00,1.20,7.5,13.50,1.15,8.00,16.00,1.16,8.56,16.22,1.18,7.0,17.00,1.20,7.65,16.00,1.20,9.10,18.00,1.18,7.86,15.87,1.19,9.0,18.00,1.44,2.75,1.46,2.79,1.50,2.82,1.46,2.70,1.53,2.84,-2.0,1.93,2.00,1.88,2.00,1.97,2.00,1.90,1.96,1.94,2.00,1.14,8.50,15.00,1.16,8.50,18.00,1.13,8.5,17.00,1.15,9.05,18.76,1.15,8.0,19.00,1.16,9.39,16.60,1.17,9.40,21.00,1.15,8.62,18.11,1.17,9.40,21.00,1.40,3.00,1.41,2.98,1.45,3.00,1.42,2.93,1.44,3.20,-2.25,2.02,1.91,2.00,1.90,2.05,1.93,1.99,1.87,2.02,1.96
3,E0,17/08/2024,15:00,Everton,Brighton,0,3,A,0,1,A,S Hooper,9,10,1,5,8,8,1,5,1,1,1,0,2.63,3.30,2.63,2.65,3.4,2.60,2.70,3.40,2.63,2.73,3.36,2.71,2.60,3.5,2.70,2.68,3.66,2.63,2.76,3.66,2.78,2.67,3.41,2.68,2.78,3.5,2.78,1.80,2.00,1.83,2.05,1.85,2.08,1.81,2.04,1.88,2.08,0.0,1.96,1.97,1.96,1.94,1.97,1.97,1.94,1.94,1.99,1.99,3.10,3.40,2.30,3.00,3.40,2.37,3.00,3.3,2.30,3.15,3.41,2.40,3.10,3.3,2.40,3.16,3.47,2.34,3.16,3.50,2.45,3.06,3.40,2.38,3.15,3.55,2.46,1.93,1.97,1.93,1.97,1.95,2.00,1.89,1.96,1.94,2.04,0.25,1.87,2.06,1.86,2.07,1.92,2.10,1.83,2.04,1.88,2.11
4,E0,17/08/2024,15:00,Newcastle,Southampton,1,0,H,1,0,H,C Pawson,3,19,1,4,15,16,3,12,2,4,1,0,1.36,5.25,8.00,1.35,5.5,7.75,1.33,5.50,8.50,1.35,5.70,8.25,1.35,5.5,8.00,1.37,5.74,8.10,1.37,5.90,8.60,1.35,5.62,8.10,1.37,6.0,9.20,1.40,3.00,1.40,3.09,1.42,3.12,1.40,3.01,1.43,3.15,-1.5,1.98,1.95,1.94,1.96,1.99,1.98,1.93,1.93,1.94,2.01,1.40,5.00,7.00,1.39,5.25,7.25,1.36,5.0,7.50,1.42,5.30,7.26,1.40,5.0,7.50,1.39,5.34,7.90,1.44,5.75,8.00,1.39,5.27,7.33,1.43,5.50,8.20,1.44,2.75,1.46,2.85,1.46,3.05,1.43,2.84,1.49,2.98,-1.25,1.87,2.06,1.88,2.06,1.89,2.10,1.82,2.05,1.89,2.10



Data types:
Div          object
Date         object
Time         object
HomeTeam     object
AwayTeam     object
             ...   
MaxCAHA     float64
AvgCAHH     float64
AvgCAHA     float64
BFECAHH     float64
BFECAHA     float64
Length: 120, dtype: object

Missing values:


,Missing Count,Missing %
BWH,141,37.11
BWCH,141,37.11
BWA,141,37.11
BWD,141,37.11
BWCA,141,37.11
BWCD,141,37.11
WHCA,91,23.95
WHCD,91,23.95
WHH,91,23.95
WHD,91,23.95


## Step 4: Check for Arsenal Matches

Checking how Arsenal appears in the data and verifying team name consistency.


In [32]:
# Check for Arsenal in the sample file
if 'df_sample' in locals():
    # Find all unique team names
    home_teams = set(df_sample['HomeTeam'].unique())
    away_teams = set(df_sample['AwayTeam'].unique())
    all_teams = home_teams.union(away_teams)
    
    print(f"Total unique teams: {len(all_teams)}")
    print(f"\nTeams containing 'Arsenal':")
    arsenal_teams = [team for team in all_teams if 'Arsenal' in str(team)]
    if arsenal_teams:
        for team in arsenal_teams:
            print(f"   {team}")
    else:
        print("   No team with 'Arsenal' found")
    
    # Show Arsenal matches
    if arsenal_teams:
        arsenal_name = arsenal_teams[0]  # Use the first match
        arsenal_matches = df_sample[
            (df_sample['HomeTeam'] == arsenal_name) | 
            (df_sample['AwayTeam'] == arsenal_name)
        ]
        print(f"\nArsenal matches in this file: {len(arsenal_matches)}")
        print(f"\nSample Arsenal matches:")
        display(arsenal_matches[['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR']].head(10))


Total unique teams: 20

Teams containing 'Arsenal':
   Arsenal

Arsenal matches in this file: 38

Sample Arsenal matches:


,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
2,17/08/2024,Arsenal,Wolves,2,0,H
16,24/08/2024,Aston Villa,Arsenal,0,2,A
20,31/08/2024,Arsenal,Brighton,1,1,D
38,15/09/2024,Tottenham,Arsenal,0,1,A
49,22/09/2024,Man City,Arsenal,2,2,D
51,28/09/2024,Arsenal,Leicester,4,2,H
61,05/10/2024,Arsenal,Southampton,3,1,H
76,19/10/2024,Bournemouth,Arsenal,2,0,H
89,27/10/2024,Arsenal,Liverpool,2,2,D
90,02/11/2024,Newcastle,Arsenal,1,0,H


## Step 5: Load All Season Files

Loading ALL the CSV files and combining them into one big dataset.


In [33]:
# Function to load all CSV files with error handling
def load_all_seasons(data_dir):
    """
    Load all EPL season CSV files and combine them.
    Handles encoding and parsing errors for problematic files.
    
    Returns:
    - list of DataFrames (one per season)
    - list of season names
    """
    csv_files = sorted(data_dir.glob('epl-*.csv'))
    
    dataframes = []
    season_names = []
    
    for file in csv_files:
        try:
            # Extract season name from filename (e.g., 'epl-2024-25.csv' -> '2024-25')
            season = file.stem.replace('epl-', '')
            
            # Special handling for problematic files
            if '2003-04' in file.name:
                # Handle parsing error - use Python engine and skip bad lines
                df = pd.read_csv(file, engine='python', on_bad_lines='skip')
            elif '2004-05' in file.name:
                # Handle encoding error - try different encodings
                try:
                    df = pd.read_csv(file, encoding='utf-8')
                except UnicodeDecodeError:
                    try:
                        df = pd.read_csv(file, encoding='latin-1')
                    except:
                        df = pd.read_csv(file, encoding='cp1252')
            else:
                # Normal loading for other files
                df = pd.read_csv(file)
            
            # Add a column to track which season this is
            df['Season'] = season
            
            dataframes.append(df)
            season_names.append(season)
            
            print(f"Loaded {file.name}: {len(df)} matches")
            
        except Exception as e:
            print(f"Error loading {file.name}: {e}")
    
    return dataframes, season_names

# Load all files (this should now work for all 26 files!)
all_dataframes, season_names = load_all_seasons(RAW_DATA_DIR)

print(f"\nTotal seasons loaded: {len(all_dataframes)}")


Loaded epl-2000-01.csv: 380 matches
Loaded epl-2001-02.csv: 380 matches
Loaded epl-2002-03.csv: 380 matches
Loaded epl-2003-04.csv: 335 matches
Error loading epl-2004-05.csv: Error tokenizing data. C error: Expected 57 fields in line 337, saw 62

Loaded epl-2005-06.csv: 380 matches
Loaded epl-2006-07.csv: 380 matches
Loaded epl-2007-08.csv: 380 matches
Loaded epl-2008-09.csv: 380 matches
Loaded epl-2009-10.csv: 380 matches
Loaded epl-2010-11.csv: 380 matches
Loaded epl-2011-12.csv: 380 matches
Loaded epl-2012-13.csv: 380 matches
Loaded epl-2013-14.csv: 380 matches
Loaded epl-2014-15.csv: 381 matches
Loaded epl-2015-16.csv: 380 matches
Loaded epl-2016-17.csv: 380 matches
Loaded epl-2017-18.csv: 380 matches
Loaded epl-2018-19.csv: 380 matches
Loaded epl-2019-20.csv: 380 matches
Loaded epl-2020-21.csv: 380 matches
Loaded epl-2021-22.csv: 380 matches
Loaded epl-2022-23.csv: 380 matches
Loaded epl-2023-24.csv: 380 matches
Loaded epl-2024-25.csv: 380 matches
Loaded epl-2025-26.csv: 120 match

## Step 6: Combine All Seasons

Combining all seasons into one master DataFrame.


In [34]:
if len(all_dataframes) > 0:
    # Combine all DataFrames
    # Note: Different seasons might have different columns, so I'll align them
    master_df = pd.concat(all_dataframes, ignore_index=True, sort=False)
    
    print(f"Combined all seasons!")
    print(f"\nMaster DataFrame shape: {master_df.shape[0]:,} rows × {master_df.shape[1]} columns")
    print(f"\nSeasons included: {sorted(master_df['Season'].unique())}")
    print(f"\nAll columns ({len(master_df.columns)} total):")
    for i, col in enumerate(master_df.columns, 1):
        print(f"   {i:2d}. {col}")
    
    print(f"\nFirst few rows:")
    display(master_df.head())
    
    print(f"\nBasic statistics:")
    print(f"   Total matches: {len(master_df):,}")
    # Date range might have mixed formats, so I'll just show sample dates
    print(f"   Sample dates: {master_df['Date'].head(3).tolist()}")
else:
    print("No data loaded!")


Combined all seasons!

Master DataFrame shape: 9,196 rows × 220 columns

Seasons included: ['2000-01', '2001-02', '2002-03', '2003-04', '2005-06', '2006-07', '2007-08', '2008-09', '2009-10', '2010-11', '2011-12', '2012-13', '2013-14', '2014-15', '2015-16', '2016-17', '2017-18', '2018-19', '2019-20', '2020-21', '2021-22', '2022-23', '2023-24', '2024-25', '2025-26']

All columns (220 total):
    1. Div
    2. Date
    3. HomeTeam
    4. AwayTeam
    5. FTHG
    6. FTAG
    7. FTR
    8. HTHG
    9. HTAG
   10. HTR
   11. Attendance
   12. Referee
   13. HS
   14. AS
   15. HST
   16. AST
   17. HHW
   18. AHW
   19. HC
   20. AC
   21. HF
   22. AF
   23. HO
   24. AO
   25. HY
   26. AY
   27. HR
   28. AR
   29. HBP
   30. ABP
   31. GBH
   32. GBD
   33. GBA
   34. IWH
   35. IWD
   36. IWA
   37. LBH
   38. LBD
   39. LBA
   40. SBH
   41. SBD
   42. SBA
   43. WHH
   44. WHD
   45. WHA
   46. Season
   47. SYH
   48. SYD
   49. SYA
   50. B365H
   51. B365D
   52. B365A
   53. SOH
 

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Attendance,Referee,HS,AS,HST,AST,HHW,AHW,HC,AC,HF,AF,HO,AO,HY,AY,HR,AR,HBP,ABP,GBH,GBD,GBA,IWH,IWD,IWA,LBH,LBD,LBA,SBH,SBD,SBA,WHH,WHD,WHA,Season,SYH,SYD,SYA,B365H,B365D,B365A,SOH,SOD,SOA,GB>2.5,GB<2.5,B365>2.5,B365<2.5,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,GBAHH,GBAHA,GBAH,LBAHH,LBAHA,LBAH,B365AHH,B365AHA,B365AH,BWH,BWD,BWA,SJH,SJD,SJA,VCH,VCD,VCA,Bb1X2,BbMxH,BbAvH,BbMxD,BbAvD,BbMxA,BbAvA,BbOU,BbMx>2.5,BbAv>2.5,BbMx<2.5,BbAv<2.5,BbAH,BbAHh,BbMxAHH,BbAvAHH,BbMxAHA,BbAvAHA,BSH,BSD,BSA,PSH,PSD,PSA,PSCH,PSCD,PSCA,Time,MaxH,MaxD,MaxA,AvgH,AvgD,AvgA,P>2.5,P<2.5,Max>2.5,Max<2.5,Avg>2.5,Avg<2.5,AHh,PAHH,PAHA,MaxAHH,MaxAHA,AvgAHH,AvgAHA,B365CH,B365CD,B365CA,BWCH,BWCD,BWCA,IWCH,IWCD,IWCA,WHCH,WHCD,WHCA,VCCH,VCCD,VCCA,MaxCH,MaxCD,MaxCA,AvgCH,AvgCD,AvgCA,B365C>2.5,B365C<2.5,PC>2.5,PC<2.5,MaxC>2.5,MaxC<2.5,AvgC>2.5,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA,BFH,BFD,BFA,1XBH,1XBD,1XBA,BFEH,BFED,BFEA,BFE>2.5,BFE<2.5,BFEAHH,BFEAHA,BFCH,BFCD,BFCA,1XBCH,1XBCD,1XBCA,BFECH,BFECD,BFECA,BFEC>2.5,BFEC<2.5,BFECAHH,BFECAHA,BFDH,BFDD,BFDA,BMGMH,BMGMD,BMGMA,BVH,BVD,BVA,CLH,CLD,CLA,BFDCH,BFDCD,BFDCA,BMGMCH,BMGMCD,BMGMCA,BVCH,BVCD,BVCA,CLCH,CLCD,CLCA,LBCH,LBCD,LBCA
0,E0,19/08/00,Charlton,Man City,4.0,0.0,H,2.0,0.0,H,20043.0,Rob Harris,17.0,8.0,14.0,4.0,2.0,1.0,6.0,6.0,13.0,12.0,8.0,6.0,1.0,2.0,0.0,0.0,10.0,20.0,2.00,3.0,3.2,2.20,2.9,2.7,2.20,3.25,2.75,2.20,3.25,2.88,2.10,3.2,3.10,2000-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,E0,19/08/00,Chelsea,West Ham,4.0,2.0,H,1.0,0.0,H,34914.0,Graham Barber,17.0,12.0,10.0,5.0,1.0,0.0,7.0,7.0,19.0,14.0,2.0,3.0,1.0,2.0,0.0,0.0,10.0,20.0,1.47,3.4,5.2,1.60,3.2,4.2,1.50,3.40,6.00,1.50,3.60,6.00,1.44,3.6,6.50,2000-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,E0,19/08/00,Coventry,Middlesbrough,1.0,3.0,A,1.0,1.0,D,20624.0,Barry Knight,6.0,16.0,3.0,9.0,0.0,1.0,8.0,4.0,15.0,21.0,1.0,3.0,5.0,3.0,1.0,0.0,75.0,30.0,2.15,3.0,3.0,2.20,2.9,2.7,2.25,3.20,2.75,2.30,3.20,2.75,2.30,3.2,2.62,2000-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Basic statistics:
   Total matches: 9,196
   Sample dates: ['19/08/00', '19/08/00', '19/08/00']


## Step 7: Filter for Arsenal Matches
Filter to only keep matches where Arsenal played (either home or away).


In [35]:
if 'master_df' in locals():
    # Find all possible Arsenal team name variations
    all_home_teams = set(master_df['HomeTeam'].dropna().unique())
    all_away_teams = set(master_df['AwayTeam'].dropna().unique())
    all_teams = all_home_teams.union(all_away_teams)
    
    # Find Arsenal variations
    arsenal_variations = [team for team in all_teams if 'Arsenal' in str(team)]
    
    print(f"Found Arsenal name variations: {arsenal_variations}")
    
    if arsenal_variations:
        # Use the most common one (usually just 'Arsenal')
        arsenal_name = arsenal_variations[0]
        print(f"\nUsing '{arsenal_name}' as Arsenal team name")
        
        # Filter for Arsenal matches
        arsenal_matches = master_df[
            (master_df['HomeTeam'] == arsenal_name) | 
            (master_df['AwayTeam'] == arsenal_name)
        ].copy()
        
        print(f"\nTotal Arsenal matches found: {len(arsenal_matches):,}")
        print(f"\nMatches per season:")
        matches_per_season = arsenal_matches['Season'].value_counts().sort_index()
        display(matches_per_season)
        
        print(f"\nSample Arsenal matches:")
        display(arsenal_matches[['Season', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR']].head(10))
    else:
        print("Could not find Arsenal in team names!")


Found Arsenal name variations: ['Arsenal']

Using 'Arsenal' as Arsenal team name

Total Arsenal matches found: 919

Matches per season:


Season
2000-01    38
2001-02    38
2002-03    38
2003-04    33
2005-06    38
2006-07    38
2007-08    38
2008-09    38
2009-10    38
2010-11    38
2011-12    38
2012-13    38
2013-14    38
2014-15    38
2015-16    38
2016-17    38
2017-18    38
2018-19    38
2019-20    38
2020-21    38
2021-22    38
2022-23    38
2023-24    38
2024-25    38
2025-26    12
Name: count, dtype: int64


Sample Arsenal matches:


,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
7,2000-01,19/08/00,Sunderland,Arsenal,1.0,0.0,H
10,2000-01,21/08/00,Arsenal,Liverpool,2.0,0.0,H
19,2000-01,26/08/00,Arsenal,Charlton,5.0,3.0,H
34,2000-01,06/09/00,Chelsea,Arsenal,2.0,2.0,D
39,2000-01,09/09/00,Bradford,Arsenal,1.0,1.0,D
49,2000-01,16/09/00,Arsenal,Coventry,2.0,1.0,H
62,2000-01,23/09/00,Ipswich,Arsenal,1.0,1.0,D
76,2000-01,01/10/00,Arsenal,Man United,1.0,0.0,H
79,2000-01,14/10/00,Arsenal,Aston Villa,1.0,0.0,H
96,2000-01,21/10/00,West Ham,Arsenal,1.0,2.0,A


## Step 8: Summary Statistics

Some basic statistics about our data.


In [36]:
if 'arsenal_matches' in locals():
    print("ARSENAL MATCHES SUMMARY\n")
    print("=" * 50)
    
    # Total matches
    print(f"\nTotal matches: {len(arsenal_matches):,}")
    
    # Home vs Away
    home_matches = len(arsenal_matches[arsenal_matches['HomeTeam'] == arsenal_name])
    away_matches = len(arsenal_matches[arsenal_matches['AwayTeam'] == arsenal_name])
    print(f"   Home matches: {home_matches}")
    print(f"   Away matches: {away_matches}")
    
    # Results distribution
    print(f"\nResults distribution:")
    results = arsenal_matches['FTR'].value_counts()
    for result, count in results.items():
        pct = (count / len(arsenal_matches) * 100)
        result_name = {'H': 'Win (Home)', 'A': 'Loss (Away)', 'D': 'Draw'}.get(result, result)
        print(f"   {result_name}: {count} ({pct:.1f}%)")
    
    # Goals
    print(f"\nGoals:")
    # Calculate goals scored and conceded
    goals_scored = []
    goals_conceded = []
    
    for _, row in arsenal_matches.iterrows():
        if row['HomeTeam'] == arsenal_name:
            goals_scored.append(row['FTHG'])
            goals_conceded.append(row['FTAG'])
        else:
            goals_scored.append(row['FTAG'])
            goals_conceded.append(row['FTHG'])
    
    print(f"   Goals scored: {sum(goals_scored)} (avg: {np.mean(goals_scored):.2f} per match)")
    print(f"   Goals conceded: {sum(goals_conceded)} (avg: {np.mean(goals_conceded):.2f} per match)")
    print(f"   Goal difference: {sum(goals_scored) - sum(goals_conceded)}")
    
    print(f"\nDate range: {arsenal_matches['Date'].min()} to {arsenal_matches['Date'].max()}")


ARSENAL MATCHES SUMMARY


Total matches: 919
   Home matches: 459
   Away matches: 460

Results distribution:
   Win (Home): 438 (47.7%)
   Loss (Away): 270 (29.4%)
   Draw: 211 (23.0%)

Goals:
   Goals scored: 1734.0 (avg: 1.89 per match)
   Goals conceded: 949.0 (avg: 1.03 per match)
   Goal difference: 785.0

Date range: 01/01/01 to 31/12/2023


## Step 9: Check for Data Quality Issues

I'm identifying any problems with the data that I'll need to fix.


In [37]:
if 'master_df' in locals():
    print("DATA QUALITY CHECK\n")
    print("=" * 50)
    
    # 1. Missing values
    print("\n1. Missing Values:")
    missing = master_df.isnull().sum()
    missing_pct = (missing / len(master_df) * 100).round(2)
    
    columns_with_missing = missing[missing > 0].sort_values(ascending=False)
    if len(columns_with_missing) > 0:
        print(f"   {len(columns_with_missing)} columns have missing values:")
        for col, count in columns_with_missing.head(10).items():
            print(f"      - {col}: {count:,} ({missing_pct[col]}%)")
    else:
        print("   No missing values!")
    
    # 2. Check for EPL only (Div should be E0)
    print("\n2. Division Check:")
    if 'Div' in master_df.columns:
        divisions = master_df['Div'].value_counts()
        print(f"   Divisions found: {dict(divisions)}")
        if len(divisions) == 1 and divisions.index[0] == 'E0':
            print("   All matches are EPL (E0)")
        else:
            print("   Multiple divisions found - may need filtering")
    
    # 3. Date format check
    print("\n3. Date Format Check:")
    print(f"   Sample dates: {master_df['Date'].head(5).tolist()}")
    print("   Dates may need to be converted to datetime format")
    
    # 4. Team name consistency
    print("\n4. Team Name Consistency:")
    all_teams = set(master_df['HomeTeam'].dropna().unique()) | set(master_df['AwayTeam'].dropna().unique())
    print(f"   Total unique teams: {len(all_teams)}")
    print("   Need to check for team name variations (e.g., 'Man United' vs 'Manchester United')")


DATA QUALITY CHECK


1. Missing Values:
   219 columns have missing values:
      - Unnamed: 52: 9,196 (100.0%)
      - Unnamed: 51: 9,196 (100.0%)
      - Unnamed: 50: 9,196 (100.0%)
      - Unnamed: 49: 9,196 (100.0%)
      - Unnamed: 48: 9,196 (100.0%)
      - BMGMA: 9,076 (98.7%)
      - CLD: 9,076 (98.7%)
      - CLH: 9,076 (98.7%)
      - BVA: 9,076 (98.7%)
      - BVD: 9,076 (98.7%)

2. Division Check:
   Divisions found: {'E0': np.int64(9195)}
   All matches are EPL (E0)

3. Date Format Check:
   Sample dates: ['19/08/00', '19/08/00', '19/08/00', '19/08/00', '19/08/00']
   Dates may need to be converted to datetime format

4. Team Name Consistency:
   Total unique teams: 46
   Need to check for team name variations (e.g., 'Man United' vs 'Manchester United')


## Next Steps

Based on this exploration, we now know:
1. ✅ Data structure and columns
2. ✅ How many Arsenal matches we have
3. ✅ What data quality issues exist

